In [1]:
import os
import pandas as pd
import numpy as np
import CorrHelpers as ch

In [2]:
baseDir = os.getcwd()
dataLocation = 'data/Master_Data.csv'
dataFilePath = os.path.join(baseDir, dataLocation)

In [3]:
roughData = pd.read_csv(dataFilePath, index_col=0)

In [4]:
# Clean data by removing stocks that dont have history
# to 2005
masterData = roughData.dropna(axis=1)

In [5]:
byStockAndYear = ch.ByStockAndYear(masterData)

In [140]:
byStockAndYear['MSFT'].keys()

Int64Index([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
            2016, 2017, 2018],
           dtype='int64')

In [ ]:
byStockAnd

In [8]:
# Drop 2019 data
for _, data in byStockAndYear.items():
    del data[2019]

# Create Rolling Correlation Data

In [9]:
rollCorr_60 = ch.RollCorr(byStockAndYear, period=60)

In [12]:
import csv

In [28]:
# Save data to file.
Saver(rollCorr_60)

In [27]:
def Saver(dictex):
    '''
    Code to save rolling correlation data to a csv file.
    from stack overflow: 
    https://stackoverflow.com/questions/50786266/writing-dictionary-of-dataframes-to-file
    '''
    for key, val in dictex.items():
        val.to_csv(f'PrivateData/RollCorr_60/RollCorr60_{key}.csv')
    with open('PrivateData/RollCorr_60/keys.txt', 'w') as f:
        f.write(str(list(dictex.keys())))
def Loader():
    with open('PrivateData/RollCorr_60/keys.txt', 'r') as f:
        keys = eval(f.read())
    dictex = {}
    for key in keys:
        dictex[key] = pd.read_csv(f'PrivateData/RollCorr_60/RollCorr60_{key}.csv')
    return dictex

# Create Seasonal Correlation Data

In [31]:
seasonalCorr_60 = ch.SeasonCorrTest(rollCorr_60, dropNum=10, n=0.75)

# Filter Seasonal Correlation Data for days with High Correlation

In [33]:
daysWithHighCorr_60 = ch.HighCorrDays(seasonalCorr_60)

# Calculate % Change for periods of high correlation

In [37]:
pctChangeResults_60 = ch.PctReturnForDays(daysWithHighCorr_60, byStockAndYear, periods=60)

# Executive Summary

In [38]:
execSummary_60 = ch.ExecSummaryCorr(pctChangeResults_60)

In [79]:
for stock, days in execSummary_60.items():
    for day, data in days.items():
        dayValue = int(day[3:])
        if int(day[3:]) > 200:
            if (data['NumPos'] > 10) or (data['NumPos'] < 4):
                with open('PrivateData/ExecSummary_60.txt', 'a') as f:
                    f.write(f'{stock} {day} start:{dayValue-60} \n {data}\n')
                    f.write('\n')
                    


In [178]:
test = {}
key = ['a','b','c','d']
items=np.array([x for x in range(10)])
items = [[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20],[21,22,23,24,25,26,27,28,29,30],
         [31,32,33,34,35,36,37,38,39,40]]
mult=0
for k in key:
    test[k] = items[mult]
    mult+=1

In [179]:
test

{'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'b': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 'c': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
 'd': [31, 32, 33, 34, 35, 36, 37, 38, 39, 40]}

In [ ]:
newdict = {}
keys = list(test.keys())
year = 1
for k, v in enumerate(keys):
    try:
        print(k)
        a = test[keys[k]][5:]
        b = test[keys[k+1]][:5]
        print(a)
        print(b)
        newData = a+b
        print(newData)
        newdict[v] = newData
    except:
        print('End Of List')
        break

In [ ]:
newdict

In [201]:
shortData = pd.read_csv('data/short_clean_closepx.csv', index_col=0)
shortData = ch.ByStockAndYear(shortData)
for _, data in shortData.items():
    del data[2019]

In [259]:
shortData['MSFT'].columns

Int64Index([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
            2016, 2017, 2018],
           dtype='int64')

In [282]:
loopYear = {}
for stock, data in shortData.items():
    columns = list(data.columns)
    daysData = {}
    for i, column in enumerate(columns):
        try:
            a = data[columns[i]][150:]
            b = data[columns[i+1]][:70]
            c = pd.concat([a,b])
            daysData[f'{columns[i]}/{columns[i+1]}'] = c
        except:
            continue
    loopYear[stock] = pd.DataFrame(daysData)

In [284]:
loopYear

{'MSFT':      2005/2006  2006/2007  2007/2008  2008/2009  2009/2010  2010/2011  \
 151  27.129999  24.340000  30.000000  27.020000  23.559999  25.610001   
 152  27.350000  24.440001  29.299999  27.389999  23.420000  25.070000   
 153  26.950001  24.459999  28.709999  28.129999  23.129999  24.860001   
 154  27.270000  24.430000  28.629999  27.900000  23.530001  24.490000   
 155  27.049999  24.530001  28.270000  28.120001  23.620001  24.400000   
 156  27.129999  24.620001  28.100000  27.910000  23.690001  24.500000   
 157  26.740000  24.700001  27.809999  27.910000  23.250000  24.709999   
 158  26.950001  24.700001  28.250000  27.809999  23.580000  24.820000   
 159  26.820000  25.790001  28.260000  27.690001  23.650000  24.440001   
 160  26.719999  26.120001  28.070000  27.320000  23.670000  24.230000   
 161  26.910000  25.620001  28.219999  27.290001  24.410000  24.280001   
 162  26.870001  25.670000  28.299999  27.180000  24.639999  24.040001   
 163  26.809999  25.740000  28